# Surface Processing Tool

This notebook reads a CSV of tracker piles, applies slope limiting, cut/fill logic, and tracker sliding optimization to minimize grading.

In [1]:
import io, pandas as pd
import numpy as np
import os
import ipywidgets as widgets
import base64
import time
import math
import tempfile
import simplekml
import pyproj
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.ndimage import median_filter, gaussian_filter
from scipy.interpolate import griddata
from scipy import stats
from pyproj import CRS, Transformer, Proj
from IPython.display import display, clear_output, HTML
from ipywidgets import VBox, HBox
from scipy.interpolate import griddata
from scipy.spatial import cKDTree, distance
from scipy.signal import savgol_filter

In [2]:
# Load Data and Create Trackers
def load_data_and_create_trackers(input_csv, tolerance_widget):
    tolerance = tolerance_widget.value

    #Loads inputed CSV into tool
    df = pd.read_csv(input_csv)

    #If the CSV is all in 1 column, split it into seperate columns according to headers
    if len(df.columns) == 1 and ',' in df.columns[0]:
        df = pd.read_csv(input_csv, header=None)
        df = df[0].str.split(',', expand=True)
        df.columns = df.iloc[0]
        df = df[1:]

    #Standardizes Column Names
    df.columns = df.columns.str.strip()
    df = df.rename(columns={'Pile In Tracker': 'PileInTracker', 'Elevation': 'GroundElevation'})

    #Converts values to numbers, if it can't, remove them
    df['PileInTracker'] = pd.to_numeric(df['PileInTracker'], errors='coerce')
    df['Northing'] = pd.to_numeric(df['Northing'], errors='coerce')
    df['GroundElevation'] = pd.to_numeric(df['GroundElevation'], errors='coerce')

    #Group data into individual tracker groups
    tracker_groups = []#List to hold all trackers
    current_group = []#Temporary list to build the current tracker

    #Iterate through all rows and divide into tracker groups
    for idx, row in df.iterrows():
        if row['PileInTracker'] == 1 and current_group:
            tracker_groups.append(current_group)
            current_group = []
        current_group.append(row.to_dict())

    if current_group:
        tracker_groups.append(current_group)

    return df, tracker_groups

In [3]:
#Step 1: TOP (Top of Pile) Calculation with Slope Limiting
def process_tracker_group_final(group, min_top, max_top, max_slope_deg):

    #group = sorted(group, key = lambda p: p['Northing'])
    #max_slope = math.tan(math.radians(max_slope_deg))
    max_slope = math.radians(max_slope_deg)
    
    #Get ground elevation and northing at the first and last pile in the tracker
    z_start = group[0]['GroundElevation']
    z_end = group[-1]['GroundElevation']
    y_start = group[0]['Northing']
    y_end = group[-1]['Northing']

    #Calculate ground slope between first and last pile
    actual_slope = (z_end - z_start) / (y_end - y_start) if y_end != y_start else 0

    #If slope is within the allowed max slope, use it as-is. If not, limit slope to the max allowable
    if abs(actual_slope) <= max_slope:
        slope_used = actual_slope
        anchor_y = y_start
        anchor_z = z_start
    else:
        slope_used = min(max_slope, max(-max_slope, actual_slope))
        center_index = len(group) // 2
        anchor_y = group[center_index]['Northing']
        anchor_z = group[center_index]['GroundElevation']

    #Calculate average desired pile top height
    avg_top = (min_top + max_top) / 2

    # Calculate slope-adjusted surface for each pile
    for pt in group:
        offset = pt['Northing'] - anchor_y
        slope_adjusted_z = anchor_z + slope_used * offset
        pt['SlopeLimitedElevation'] = slope_adjusted_z
        pt['SOP'] = slope_adjusted_z + avg_top

    return group


In [4]:
#Step 2: Initial Cut/Fill
def apply_initial_cut_fill(df, min_top, max_top, tolerance):
    df = df.copy()

    # Compute pile top limits
    df['LowerLimit'] = df['GroundElevation'] + min_top + tolerance
    df['UpperLimit'] = df['GroundElevation'] + max_top - tolerance

    # Compute cut/fill based on SOP (pre-shift)
    df['Cut'] = np.maximum(0, df['SOP'] - df['UpperLimit'])
    df['Fill'] = np.maximum(0, df['LowerLimit'] - df['SOP'])

    # Simulate grading before shifting
    df['ProposedGround'] = df['GroundElevation'] + df['Fill'] - df['Cut']

    return df


In [5]:
#Step 3: Slide the Tracker Vertically to Minimize Grading and Shift
def slide_tracker_min_grading_min_shift(df, min_top, max_top, tolerance, step_size, max_slope_deg):
    max_slope = math.tan(math.radians(max_slope_deg))
    max_shift = max_top - min_top
    step = step_size
    shift_range = (-max_shift, max_shift)
    best_shift = 0
    min_grading = float('inf')
    shifts = np.arange(shift_range[0], shift_range[1] + step, step)

    df['LowerLimit'] = df['GroundElevation'] + min_top - tolerance
    df['UpperLimit'] = df['GroundElevation'] + max_top + tolerance

    # Test each potential vertical shift
    for shift in shifts:
        # Apply vertical shift to the slope-limited surface and add average pile height
        shifted_sop = df['SlopeLimitedElevation'] + shift + (min_top + max_top) / 2

        cut = np.maximum(0, shifted_sop - df['UpperLimit'])
        fill = np.maximum(0, df['LowerLimit'] - shifted_sop)
        grading = cut.sum() + fill.sum()

        #Get SOP at first and last pile after shift
        first = shifted_sop.iloc[0]
        last = shifted_sop.iloc[-1]

        # Check if shifted SOP still keeps edge piles within allowable limits
        first_ok = (first >= df['LowerLimit'].iloc[0]) and (first <= df['UpperLimit'].iloc[0])
        last_ok = (last >= df['LowerLimit'].iloc[-1]) and (last <= df['UpperLimit'].iloc[-1])

        # Only consider this shift if both edge piles remain within limits
        if first_ok and last_ok:
            #Calculate cut/fill required for this shift
            cut = np.maximum(0, shifted_sop - df['UpperLimit'])
            fill = np.maximum(0, df['LowerLimit'] - shifted_sop)
            grading = cut.sum() + fill.sum()  # Total grading effort = sum of cut and fill

            # Keep the shift if it reduces grading, or ties but is a smaller shift
            if grading < min_grading or (grading == min_grading and abs(shift) < abs(best_shift)):
                min_grading = grading
                best_shift = shift

    # Once optimal shift is found, apply it to the SOP
    df['SOP_Slid'] = df['SlopeLimitedElevation'] + best_shift + (min_top + max_top) / 2
    
    # Recalculate cut/fill based on the shifted SOP
    df['Cut_Slid'] = np.maximum(0, df['SOP_Slid'] - df['UpperLimit'])
    df['Fill_Slid'] = np.maximum(0, df['LowerLimit'] - df['SOP_Slid'])

    # Compute new proposed ground surface after applying cut/fill
    df['ProposedGround_Slid'] = df['GroundElevation'] - df['Fill_Slid'] + df['Cut_Slid']

    # Record the vertical shift applied
    df['VerticalShift'] = best_shift

   # df['Delta'] = df['ProposedGround_Slid'] - df['GroundElevation']
  #  print(df[['PileID', 'GroundElevation', 'ProposedGround_Slid', 'Delta']].head(10))


    return df  # Return updated DataFrame for this tracker


In [6]:
#Run full pipeline for all trackers
def run_full_pipeline(tracker_groups, min_top, max_top, tolerance, step_size, max_slope_deg):
    all_results = []
    with out:
        #Initialize Progress bar
        progress = widgets.FloatProgress(value = 0, min = 0, max = 1, bar_style = 'info', description = 'Progress: ')
        label0 = widgets.HTML("<span style = 'font-weight: bold;'>Analyzing Data...</span>")
        est_remaining = 1
        label1 = widgets.Label(f"{int(progress.value*100)}% | ~{int(est_remaining)}s remaining", layout = widgets.Layout(align_items = 'center'))
        display(widgets.VBox([label0, widgets.HBox([progress, label1])]))
        total = len(tracker_groups)
        start_time = time.time()
        i = 1
    
        for group in tracker_groups:
            step_start = time.time()
            
            #Step 1
            group = process_tracker_group_final(group, min_top, max_top, max_slope_deg)
            df = pd.DataFrame(group)  # Convert list of dicts to DataFrame

            #Step 2
            df_diag = apply_initial_cut_fill(df, min_top, max_top, tolerance)

            #Step 3
            df_final = slide_tracker_min_grading_min_shift(df, min_top, max_top, tolerance, step_size, max_slope_deg)
            all_results.append(df_final)

            #Update Progress Bar
            progress.value =  (i + 1) / total
            elapsed = time.time() - start_time
            avg_time = elapsed / (i + 1)
            est_remaining = avg_time * (total - (i + 1))
            label1.value = f"{int(progress.value*100)}% | ~{int(est_remaining)}s remaining"
            i=i+1
            
        #Update Progress Bar to complete when complete lol
        label0.value = "<span style = 'font-weight: bold;'>Complete!</span>"
        progress.bar_style = 'success'
    
    final_df = pd.concat(all_results, ignore_index=True)
    return final_df

In [7]:
#Export to CSV
def export_csv(final_df, project_name):
    #Creates CSV from final_df
    output_path = f"{project_name} Proposed.csv"
    output = final_df[['PileID', 'PileInTracker', 'Northing', 'Easting', 'ProposedGround_Slid']].copy()
    output = output.rename(columns={'ProposedGround_Slid': 'Elevation'})
    output.to_csv(output_path, index=False)

    #Creates link to download CSV
    csv_data = output.to_csv(index=False)
    b64 = base64.b64encode(csv_data.encode()).decode()
    href = f'<a download="{output_path}" href="data:text/csv;base64,{b64}" target="_blank" style="font-size:16px; text-decoration:none;">⬇️ Download {output_path}</a>'

    #Shows link to download CSV
    with out2:
        display(HTML(href))
    
    return output_path

In [8]:
def load_surface(csv_path, rename_map):
    #Reload input CSV
    df = pd.read_csv(csv_path)

    #If the CSV is all in 1 column, split it into seperate columns according to headers
    if len(df.columns) == 1 and ',' in df.columns[0]:
        df = pd.read_csv(csv_path, header=None)
        df = df[0].str.split(',', expand=True)
        df.columns = df.iloc[0].str.strip().str.replace('"', '')
        df = df[1:].reset_index(drop=True)

    #Clean headers
    df.columns = df.columns.str.strip().str.replace('"', '')
                
    df = df.rename(columns=rename_map)
    return df

In [9]:
# Compare proposed surface to existing surface
def compare_surfaces(input_csv, project_name):
    existing_csv = input_csv
    proposed_csv = f"{project_name} Proposed.csv"
    area_per_point_sqft = 25
    
    # Load surfaces
    df_existing = load_surface(existing_csv, {"Elevation": "ExistingElevation", "Pile In Tracker": "PileInTracker"})
    df_proposed = load_surface(proposed_csv, {"Elevation": "ProposedElevation"})

    df_existing['PileID'] = pd.to_numeric(df_existing['PileID'], errors='coerce').astype('Int64')
    df_proposed['PileID'] = pd.to_numeric(df_proposed['PileID'], errors='coerce').astype('Int64')

    for col in ['PileInTracker', 'Northing', 'Easting']:
        df_existing[col] = pd.to_numeric(df_existing[col], errors='coerce')
        df_proposed[col] = pd.to_numeric(df_proposed[col], errors='coerce')

    merge_keys = ["PileID", "PileInTracker", "Northing", "Easting"]

    df_combined = pd.merge(df_existing, df_proposed, on=merge_keys, how="inner")
    df_combined["DeltaElevation"] = (df_combined["ProposedElevation"].astype(float)- df_combined["ExistingElevation"].astype(float))

    df_filtered = df_combined.copy()

    Q1 = df_filtered["DeltaElevation"].quantile(0.05)
    Q4 = df_filtered["DeltaElevation"].quantile(0.95)
    IQR = Q4 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q4 + 1.5 * IQR

    df_filtered = df_filtered[(df_filtered["DeltaElevation"] >= lower_bound)& (df_filtered["DeltaElevation"] <= upper_bound)]

    removed_outliers = len(df_combined) - len(df_filtered)

    df_filtered["Cut"] = np.where(df_filtered["DeltaElevation"] < 0, -df_filtered["DeltaElevation"], 0)
    df_filtered["Fill"] = np.where(df_filtered["DeltaElevation"] > 0, df_filtered["DeltaElevation"], 0)

    cut_volume_ft3 = df_filtered['Cut'].sum() * area_per_point_sqft
    fill_volume_ft3 = df_filtered['Fill'].sum() * area_per_point_sqft
    cut_volume_yd3 = cut_volume_ft3 / 27
    fill_volume_yd3 = fill_volume_ft3 / 27

    with out2:
        print(f"Merged rows: {len(df_combined)} of {len(df_existing)} existing")
        print("\n📊 Delta Elevation stats (outliers removed):")
        print(df_filtered['DeltaElevation'].describe())
        print(f"\nTotal cut: {cut_volume_ft3:.2f} ft³ ({cut_volume_yd3:.2f} yd³)")
        print(f"Total fill: {fill_volume_ft3:.2f} ft³ ({fill_volume_yd3:.2f} yd³)")

        df_filtered.to_csv("df_combined_filtered.csv", index=False)

    return {"df_combined": df_combined,"df_filtered": df_filtered,"cut_volume_ft3": cut_volume_ft3,"fill_volume_ft3": fill_volume_ft3,"cut_volume_yd3": cut_volume_yd3,"fill_volume_yd3": fill_volume_yd3,"df_proposed": df_proposed}


In [10]:
def plot_heatmap(df_combined, cut_volume_ft3, fill_volume_ft3, cut_volume_yd3, fill_volume_yd3):
    with out3:
        lowest = df_combined['DeltaElevation'].quantile(0.05)
        lowest = math.floor(lowest)
        highest = df_combined['DeltaElevation'].quantile(0.95)
        highest = math.ceil(highest)

        if abs(lowest) > abs(highest):
            top_extreme = abs(lowest)
            bottom_extreme = lowest
        else:
            top_extreme = highest
            bottom_extreme = highest * -1

        ft3_strings = [f"{cut_volume_ft3:.2f}", f"{fill_volume_ft3:.2f}"]
        yd3_strings = [f"{cut_volume_yd3:.2f}", f"{fill_volume_yd3:.2f}"]

        type_width = max(len("Type"), len("Cut"), len("Fill")) * 10  # pixels per char
        ft3_width = max(len(s) for s in ft3_strings + ["ft³"]) * 10
        yd3_width = max(len(s) for s in yd3_strings + ["yd³"]) * 10 
    
        df_combined['DeltaElevationForViz'] = df_combined['DeltaElevation'].clip(lowest, highest)

        for col in ['Easting', 'Northing', 'DeltaElevationForViz']:
            df_combined[col] = pd.to_numeric(df_combined[col], errors='coerce')

        df_combined = df_combined.dropna(subset=['Easting', 'Northing', 'DeltaElevationForViz'])
    
        x = df_combined['Easting'].values
        y = df_combined['Northing'].values
        z = df_combined['DeltaElevationForViz'].values
        points = np.column_stack((x, y))

        grid_x, grid_y = np.meshgrid(np.linspace(x.min(), x.max(), 500), np.linspace(y.min(), y.max(), 500))
        grid_z = griddata(points, z, (grid_x, grid_y), method='cubic')
        grid_z = np.where(np.isnan(grid_z), None, grid_z)

        gap_factor = 110
        sample = points[np.random.choice(len(points), size=min(2000, len(points)), replace=False)]
        pdists = distance.pdist(sample)
        median_spacing = np.median(pdists)
        threshold = median_spacing * (1 / gap_factor)
        
        tree = cKDTree(points)
        distances, _ = tree.query(np.column_stack((grid_x.ravel(), grid_y.ravel())), k=1)
        distances = distances.reshape(grid_x.shape)
        grid_z[distances > threshold] = None

        fig = go.FigureWidget(make_subplots(rows=1, cols=2,column_widths=[0.75, 0.25],specs=[[{"type": "heatmap"}, {"type": "table"}]],subplot_titles=("Cut/Fill Heatmap", "Earthwork Summary")))

        fig.add_trace(go.Heatmap(z=grid_z,x=np.linspace(x.min(), x.max(), 300),y=np.linspace(y.min(), y.max(), 300),colorscale='RdYlGn',zmid=0,zmin=bottom_extreme,zmax=top_extreme,colorbar=dict(title="Δ Elevation (ft)")),row=1, col=1)

        fig.update_yaxes(scaleanchor="x", row=1, col=1, showticklabels=False)
        fig.update_xaxes(showticklabels=False)

        fig.add_trace(go.Table(header=dict(values=["Type", "ft³", "yd³"], fill_color="lightgrey", align="left"),cells=dict(values=[["Cut", "Fill"], ft3_strings, yd3_strings],),columnwidth=[type_width, ft3_width, yd3_width] ),row=1, col=2)

        fig.update_layout(title="Grading Heatmap and Volume Summary",height=700,width=1100,plot_bgcolor='white')

        display(fig)

In [11]:
def clean_grid(g):
    g = np.array(g, dtype=np.float64)
    g[~np.isfinite(g)] = None  # remove infs
    g = np.ma.filled(g, None)  # unmask
    g = np.where(np.isnan(g), None, g)  # convert to None
    return g

In [12]:
def plot_change_in_terrain(df, value_col='DeltaElevation', x_col='Easting', y_col='Northing', grid_res = 500, mask_distance = 1500):
    with out4:
        # Convert columns to numeric and drop NaNs
        for col in [x_col, y_col, value_col]:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        df = df.dropna(subset=[x_col, y_col, value_col])

        lowest = df[value_col].quantile(0.05)
        lowest = math.floor(lowest)
        highest = df[value_col].quantile(0.95)
        highest = math.ceil(highest)

        if abs(lowest) > abs(highest):
            top_extreme = abs(lowest)
            bottom_extreme = lowest
        else:
            top_extreme = highest
            bottom_extreme = highest * -1
        
        x = df[x_col] - df[x_col].min()
        y = df[y_col] - df[y_col].min()
        z = df[value_col].values
        points = np.column_stack((x,y))
        
        grid_x, grid_y = np.meshgrid(np.linspace(x.min(), x.max(), grid_res), np.linspace(y.min(), y.max(), grid_res))
        grid_z = griddata((x, y), z, (grid_x, grid_y), method='cubic')

        gap_factor = 110
        sample = points[np.random.choice(len(points), size=min(2000, len(points)), replace=False)]
        pdists = distance.pdist(sample)
        median_spacing = np.median(pdists)
        threshold = median_spacing * (1 / gap_factor)

        tree = cKDTree(points)
        distances, _ = tree.query(np.c_[grid_x.ravel(), grid_y.ravel()], k = 1)
        distances = distances.reshape(grid_x.shape)
        grid_z[distances > threshold] = None

        grid_z = clean_grid(grid_z)
        grid_z = np.array(grid_z, dtype = float)
        grid_z[~np.isfinite(grid_z)] = None
        grid_z = np.where(np.isnan(grid_z), None, grid_z)
        grid_z[distances > threshold] = None

        fig = go.FigureWidget(data=[go.Surface(z=grid_z, x=grid_y, y=grid_x, colorscale='RdYlGn', cmin = bottom_extreme, cmax = top_extreme, cmid = 0)])

        fig.update_layout(title="3D Grading Heatmap", scene_aspectmode='manual', scene_aspectratio=dict(x=1, y=1, z=0.1), scene=dict(xaxis_title=x_col, yaxis_title=y_col, zaxis_title=value_col), height=600, margin=dict(l=0, r=0, t=40, b=0))

        display(fig)

In [13]:
def get_shared_zscale(df_existing, df_proposed, existing_col='ExistingElevation', proposed_col='Elevation'):

    # Convert to numeric safely in case of strings
    z_existing = pd.to_numeric(df_existing[existing_col], errors='coerce').dropna()
    z_proposed = pd.to_numeric(df_proposed[proposed_col], errors='coerce').dropna()

    z_min = min(z_existing.min(), z_proposed.min())
    z_max = max(z_existing.max(), z_proposed.max())

    return z_min, z_max


In [14]:
def plot_proposed_ground(df, df_e, value_col='Elevation', x_col='Easting', y_col='Northing', grid_res=500, mask_distance=1500):
    with out5:
        # Convert to numeric safely
        df[x_col] = pd.to_numeric(df[x_col], errors="coerce")
        df[y_col] = pd.to_numeric(df[y_col], errors="coerce")
        df[value_col] = pd.to_numeric(df[value_col], errors="coerce")
        df = df.dropna(subset=[x_col, y_col, value_col])
        
        # Local coordinates to avoid huge numbers
        x = df[x_col] - df[x_col].min()
        y = df[y_col] - df[y_col].min()
        z = df[value_col]
        points = np.column_stack((x,y))

        # Interpolate onto grid
        grid_x, grid_y = np.meshgrid(np.linspace(x.min(), x.max(), grid_res), np.linspace(y.min(), y.max(), grid_res))
        grid_z = griddata((x, y), z, (grid_x, grid_y), method="cubic")
        grid_z = np.where(np.isnan(grid_z), None, grid_z)

        gap_factor = 110
        sample = points[np.random.choice(len(points), size=min(2000, len(points)), replace=False)]
        pdists = distance.pdist(sample)
        median_spacing = np.median(pdists)
        threshold = median_spacing * (1 / gap_factor)

        # Mask points far from data using cKDTree
        tree = cKDTree(points)
        distances, _ = tree.query(np.c_[grid_x.ravel(), grid_y.ravel()], k=1)
        distances = distances.reshape(grid_x.shape)
        grid_z[distances > threshold] = None

        grid_z = clean_grid(grid_z)
        grid_z = np.array(grid_z, dtype = float)
        grid_z = np.where(np.isnan(grid_z), None, grid_z)

        z_min, z_max = get_shared_zscale(df_e, df)

        # Create 3D surface plot
        fig = go.FigureWidget(data=[go.Surface(x=grid_x, y=grid_y, z=grid_z, cmin = z_min, cmax = z_max, colorscale="Viridis", contours={"z": {"show": True, "usecolormap": True, "project_z": True}},)])

        fig.update_layout(title=f"Proposed 3D Surface", scene=dict(xaxis_title=f"{x_col} (relative)", yaxis_title=f"{y_col} (relative)", zaxis_title="Elevation (ft)", zaxis=dict(range=[z_min, z_max]), aspectratio=dict(x=1, y=1, z=.1), ), height=700)

        display(fig)

In [15]:
def plot_existing_surface(df, df_p, value_col='ExistingElevation', x_col='Easting', y_col='Northing', mask_distance=1500, grid_res=500):
    with out6:
        # Convert to numeric safely
        df[x_col] = pd.to_numeric(df[x_col], errors="coerce")
        df[y_col] = pd.to_numeric(df[y_col], errors="coerce")
        df[value_col] = pd.to_numeric(df[value_col], errors="coerce")
        df = df.dropna(subset=[x_col, y_col, value_col])

        # Local coordinates to avoid huge numbers
        x = df[x_col] - df[x_col].min()
        y = df[y_col] - df[y_col].min()
        z = df[value_col]
        points = np.column_stack((x,y))

        # Interpolate onto grid
        grid_x, grid_y = np.meshgrid(np.linspace(x.min(), x.max(), grid_res), np.linspace(y.min(), y.max(), grid_res))
        grid_z = griddata((x, y), z, (grid_x, grid_y), method="cubic")
        grid_z = np.where(np.isnan(grid_z), None, grid_z)

        gap_factor = 110
        sample = points[np.random.choice(len(points), size=min(2000, len(points)), replace=False)]
        pdists = distance.pdist(sample)
        median_spacing = np.median(pdists)
        threshold = median_spacing * (1 / gap_factor)

        # Mask points far from data using cKDTree
        tree = cKDTree(points)
        distances, _ = tree.query(np.c_[grid_x.ravel(), grid_y.ravel()], k=1)
        distances = distances.reshape(grid_x.shape)
        grid_z[distances > threshold] = None

        grid_z = clean_grid(grid_z)
        grid_z = np.array(grid_z, dtype = float)
        grid_z = np.where(np.isnan(grid_z), None, grid_z)

        z_min, z_max = get_shared_zscale(df, df_p)

        # Create 3D surface plot
        fig = go.FigureWidget(data=[go.Surface(x=grid_x, y=grid_y, z=grid_z, cmin = z_min, cmax = z_max, colorscale="Viridis", contours={"z": {"show": True, "usecolormap": True, "project_z": True}},)])

        fig.update_layout(title=f"Existing 3D Surface", scene=dict(xaxis_title=f"{x_col} (relative)", yaxis_title=f"{y_col} (relative)", zaxis_title="Elevation (ft)", zaxis=dict(range=[z_min, z_max]), aspectratio=dict(x=1, y=1, z=0.1), ), height=700)

        display(fig)
   

In [16]:
def dot_heatmap(df_combined):
    with out7:
        lowest = df_combined['DeltaElevation'].quantile(0.05)
        lowest = math.floor(lowest)
        highest = df_combined['DeltaElevation'].quantile(0.95)
        highest = math.ceil(highest)

        if abs(lowest) > abs(highest):
            top_extreme = abs(lowest)
            bottom_extreme = lowest
        else:
            top_extreme = highest
            bottom_extreme = highest * -1
        
        df_combined['DeltaElevationForViz'] = df_combined['DeltaElevation'].clip(lowest, highest)
        for col in ['Easting', 'Northing', 'DeltaElevationForViz']:
            df_combined[col] = pd.to_numeric(df_combined[col], errors = 'coerce')
        df_combined = df_combined.dropna(subset = ['Easting', 'Northing', 'DeltaElevationForViz'])

        x = df_combined['Easting'] - df_combined['Easting'].min()
        y = df_combined['Northing'] - df_combined['Northing'].min()
        z = df_combined['DeltaElevationForViz']

        fig = go.FigureWidget(data=go.Scatter(x=x,y=y,mode='markers',marker=dict(size=2,color=z,colorscale = 'RdYlGn', cmin = bottom_extreme, cmax = top_extreme, cmid = 0, colorbar=dict(title='Change in Elevation'))))
        fig.update_layout(title='Pile Grading Heatmap',xaxis_title= 'Easting (relative)',yaxis_title= 'Northing (relative)',template='plotly_white', height=700)

        display(fig)

In [17]:
#def dot_heatmap_existing(df, value_col='ExistingElevation', x_col='Easting', y_col='Northing'):
 #   with out8:
  #      df[x_col] = pd.to_numeric(df[x_col], errors="coerce")
   #     df[y_col] = pd.to_numeric(df[y_col], errors="coerce")  
    #    df[value_col] = pd.to_numeric(df[value_col], errors="coerce")
     #   df = df.dropna(subset=[x_col, y_col, value_col])

      #  x = df[x_col] - df[x_col].min()
       # y = df[y_col] - df[y_col].min()
        #z = df[value_col]

        #fig = go.FigureWidget(data=go.Scatter(x=x,y=y,mode='markers',marker=dict(size=1,color=z,colorscale = 'RdYlGn', reversescale = True, colorbar=dict(title=value_col))))

        #fig.update_layout(title=f'{value_col} Distribution',xaxis_title=f'{x_col} (relative)',yaxis_title=f'{y_col} (relative)',template='plotly_white', height=700)

        #display(fig)


In [18]:
#def dot_heatmap_proposed(df, value_col='Elevation', x_col='Easting', y_col='Northing'):
 #   with out9:
  #      df[x_col] = pd.to_numeric(df[x_col], errors="coerce")
   #     df[y_col] = pd.to_numeric(df[y_col], errors="coerce")  
    #    df[value_col] = pd.to_numeric(df[value_col], errors="coerce")
     #   df = df.dropna(subset=[x_col, y_col, value_col])

      #  x = df[x_col] - df[x_col].min()
       # y = df[y_col] - df[y_col].min()
        #z = df[value_col]

        #fig = go.FigureWidget(data=go.Scatter(x=x,y=y,mode='markers',marker=dict(size=1,color=z,colorscale = 'RdYlGn', reversescale = True, colorbar=dict(title=value_col))))

        #fig.update_layout(title=f'{value_col} Distribution',xaxis_title=f'{x_col} (relative)',yaxis_title=f'{y_col} (relative)',template='plotly_white', height=700)

        #display(fig)


In [19]:
def plot_side_view(df, df_exported,x_col='Easting', y_col='Northing',existing_col='ExistingElevation', proposed_col='Elevation'):
    with out10:
        for col in [x_col, y_col, existing_col]:
            df[col] = pd.to_numeric(df[col], errors="coerce")
        for col in [x_col, y_col, proposed_col]:
            df_exported[col] = pd.to_numeric(df_exported[col], errors="coerce")
        df = df.dropna(subset=[x_col, y_col, existing_col])
        df_exported = df_exported.dropna(subset=[x_col, y_col, proposed_col])

        df[x_col] = df[x_col] - df[x_col].min()
        df[y_col] = df[y_col] - df[y_col].min()
        df_exported[x_col] = df_exported[x_col] - df_exported[x_col].min()
        df_exported[y_col] = df_exported[y_col] - df_exported[y_col].min()

        x_min, y_min = 0, 0
        x_max = max(df[x_col].max(), df_exported[x_col].max())
        y_max = max(df[y_col].max(), df_exported[y_col].max())
        elev_min = min(df[existing_col].min(), df_exported[proposed_col].min())
        elev_max = max(df[existing_col].max(), df_exported[proposed_col].max())

        fig_easting = go.FigureWidget()
        fig_northing = go.FigureWidget()

        northing_values = sorted(df[y_col].dropna().unique())
        easting_values = sorted(df[x_col].dropna().unique())
        
        slider_y = widgets.SelectionSlider(options=[(f"{val:.2f}", val) for val in northing_values],value=northing_values[len(northing_values)//2],description='Northing slice:',continuous_update=True,layout=widgets.Layout(width='75%'))
        slider_y.style.description_width = '95px'
        slider_x = widgets.SelectionSlider(options=[(f"{val:.2f}", val) for val in easting_values],value=easting_values[len(easting_values)//2],description='Easting slice:',continuous_update=True,layout=widgets.Layout(width='75%'))
        slider_x.style.description_width = '95px'
        
        def update_fig_easting(slice_val):
            tol = (df[y_col].max() - df[y_col].min()) / 1000
            section_df = df[np.abs(df[y_col]-slice_val)<tol].sort_values(by=x_col)
            section_exported = df_exported[np.abs(df_exported[y_col]-slice_val)<tol].sort_values(by=x_col)
            
            if len(section_df)<2 or len(section_exported)<2:
                fig_easting.data = []
                fig_easting.update_layout(title=f"No data near Y={slice_val:.2f}")
                return
                
            min_len = min(len(section_df), len(section_exported))
            section_df = section_df.iloc[:min_len]
            section_exported = section_exported.iloc[:min_len]
            x = section_df[x_col].values
            
            existing = section_df[existing_col].values
            proposed = section_exported[proposed_col].values
            max_pile_top_northing = section_exported[proposed_col].values + max_top
            min_pile_top_northing = section_exported[proposed_col].values + min_top
            
            window = min(41, len(x)//2*2+1)
            if window >= 5:
                existing = savgol_filter(existing, window_length=window, polyorder=2)
                proposed = savgol_filter(proposed, window_length=window, polyorder=2)
                max_pile_top_northing = savgol_filter(max_pile_top_northing, window_length=window, polyorder=2)
                min_pile_top_northing = savgol_filter(min_pile_top_northing, window_length=window, polyorder=2)
                
            fig_easting.data = []
            #fig_northing.add_trace(go.Scatter(x=x, y=max_pile_top_northing, mode='markers', name='Max Pile Height', line=dict(color='red')))
            #fig_northing.add_trace(go.Scatter(x=x, y=min_pile_top_northing, mode='markers', name='Min Pile Height', line=dict(color='orange')))
            fig_easting.add_trace(go.Scatter(x=x, y=existing, mode='markers', name='Existing', line=dict(color='red')))
            fig_easting.add_trace(go.Scatter(x=x, y=proposed, mode='markers', name='Proposed', line=dict(color='green')))
            fig_easting.update_layout(title=f"Easting Slice (Y ≈ {slice_val:.2f})", xaxis_title='Easting', yaxis_title='Elevation (ft)', template='plotly_white', height=500)
            fig_easting.update_xaxes(range=[x_min, x_max])
            fig_easting.update_yaxes(range=[elev_min, elev_max])

            
            mask_higher = proposed > existing
            segments = []
            current_mask = mask_higher[0]
            start_idx = 0
            for i in range(1, len(mask_higher)):
                if mask_higher[i] != current_mask:
                    segments.append((start_idx, i, current_mask))
                    start_idx = i
                    current_mask = mask_higher[i]
            segments.append((start_idx, len(mask_higher), current_mask))

            for start, end, is_higher in segments:
                seg_x = x[start:end]
                seg_existing = existing[start:end]
                seg_proposed = proposed[start:end]
                color = "rgba(0,255,0,0.4)" if is_higher else "rgba(255,0,0,0.4)"
                fig_easting.add_trace(go.Scatter(x=np.concatenate([seg_x, seg_x[::-1]]),y=np.concatenate([seg_existing, seg_proposed[::-1]]),fill='toself',
                                                 fillcolor=color,line=dict(color='rgba(255,255,255,0)'),hoverinfo='skip',showlegend=False))

        def update_fig_northing(slice_val):
            tol = (df[x_col].max() - df[x_col].min()) / 1000
            section_df = df[np.abs(df[x_col]-slice_val)<tol].sort_values(by=y_col)
            section_exported = df_exported[np.abs(df_exported[x_col]-slice_val)<tol].sort_values(by=y_col)
            
            if len(section_df)<2 or len(section_exported)<2:
                fig_northing.data = []
                fig_northing.update_layout(title=f"No data near X={slice_val:.2f}")
                return
                
            min_len = min(len(section_df), len(section_exported))
            section_df = section_df.iloc[:min_len]
            section_exported = section_exported.iloc[:min_len]
            x = section_df[y_col].values
            
            existing = section_df[existing_col].values
            proposed = section_exported[proposed_col].values
            max_pile_top_northing = section_exported[proposed_col].values + max_top
            min_pile_top_northing = section_exported[proposed_col].values + min_top
            
            window = min(41, len(x)//2*2+1)
            if window >= 5:
                existing = savgol_filter(existing, window_length=window, polyorder=2)
                proposed = savgol_filter(proposed, window_length=window, polyorder=2)
                max_pile_top_northing = savgol_filter(max_pile_top_northing, window_length=window, polyorder=2)
                min_pile_top_northing = savgol_filter(min_pile_top_northing, window_length=window, polyorder=2)

            
            fig_northing.data = []
            #fig_northing.add_trace(go.Scatter(x=x, y=max_pile_top_northing, mode='markers', name='Max Pile Height', line=dict(color='red')))
            #fig_northing.add_trace(go.Scatter(x=x, y=min_pile_top_northing, mode='markers', name='Min Pile Height', line=dict(color='orange')))
            fig_northing.add_trace(go.Scatter(x=x, y=existing, mode='markers', name='Existing', line=dict(color='red')))
            fig_northing.add_trace(go.Scatter(x=x, y=proposed, mode='markers', name='Proposed', line=dict(color='green')))
            fig_northing.update_layout(title=f"Northing Slice (X ≈ {slice_val:.2f})", xaxis_title='Northing', yaxis_title='Elevation (ft)', template='plotly_white', height=500)
            fig_northing.update_xaxes(range=[y_min, y_max])
            fig_northing.update_yaxes(range=[elev_min, elev_max])

            
            mask_higher = proposed > existing
            segments = []
            current_mask = mask_higher[0]
            start_idx = 0
            for i in range(1, len(mask_higher)):
                if mask_higher[i] != current_mask:
                    segments.append((start_idx, i, current_mask))
                    start_idx = i
                    current_mask = mask_higher[i]
            segments.append((start_idx, len(mask_higher), current_mask))

            for start, end, is_higher in segments:
                seg_x = x[start:end]
                seg_existing = existing[start:end]
                seg_proposed = proposed[start:end]
                color = "rgba(0,255,0,0.4)" if is_higher else "rgba(255,0,0,0.4)"
                fig_northing.add_trace(go.Scatter(
                    x=np.concatenate([seg_x, seg_x[::-1]]),
                    y=np.concatenate([seg_existing, seg_proposed[::-1]]),
                    fill='toself',
                    fillcolor=color,
                    line=dict(color='rgba(255,255,255,0)'),
                    hoverinfo='skip',
                    showlegend=False
                ))
                
        slider_y.observe(lambda change: update_fig_easting(change['new']), names='value')
        slider_x.observe(lambda change: update_fig_northing(change['new']), names='value')

        update_fig_easting(slider_y.value)
        update_fig_northing(slider_x.value)

        display(widgets.VBox([fig_northing, widgets.HBox([slider_x],layout = widgets.Layout(justify_content = 'center'))]))

    with out12:
        display(widgets.VBox([fig_easting, widgets.HBox([slider_y],layout = widgets.Layout(justify_content = 'center'))]))


In [20]:
#def plot_existing_vs_proposed(dfp, dfe, pro_value_col='Elevation', ex_value_col='ExistingElevation', x_col='Easting', y_col='Northing', grid_res=500, mask_distance=1500):
 #   with out11:
        # --- Clean numeric data ---
  #      for df, val_col in [(dfe, ex_value_col), (dfp, pro_value_col)]:
   #         df[x_col] = pd.to_numeric(df[x_col], errors="coerce")
    #        df[y_col] = pd.to_numeric(df[y_col], errors="coerce")
     #       df[val_col] = pd.to_numeric(df[val_col], errors="coerce")
      #      df.dropna(subset=[x_col, y_col, val_col], inplace=True)

        # --- Compute relative coordinates (to align both) ---
       # x_min = min(dfe[x_col].min(), dfp[x_col].min())
        #y_min = min(dfe[y_col].min(), dfp[y_col].min())

        #existing_x = dfe[x_col] - x_min
        #existing_y = dfe[y_col] - y_min
        #proposed_x = dfp[x_col] - x_min
        #proposed_y = dfp[y_col] - y_min

        # --- Common grid for interpolation ---
        #grid_x, grid_y = np.meshgrid(
         #   np.linspace(min(existing_x.min(), proposed_x.min()), max(existing_x.max(), proposed_x.max()), grid_res),
          #  np.linspace(min(existing_y.min(), proposed_y.min()), max(existing_y.max(), proposed_y.max()), grid_res)
        #)

        # --- Interpolate both datasets ---
        #grid_existing = griddata((existing_x, existing_y), dfe[ex_value_col], (grid_x, grid_y), method="cubic")
        #grid_proposed = griddata((proposed_x, proposed_y), dfp[pro_value_col], (grid_x, grid_y), method="cubic")

        # --- Mask out areas too far from data points ---
        #for grid_z, src_x, src_y in [(grid_existing, existing_x, existing_y), (grid_proposed, proposed_x, proposed_y)]:
         #   tree = cKDTree(np.c_[src_x, src_y])
          #  dist, _ = tree.query(np.c_[grid_x.ravel(), grid_y.ravel()], k=1)
           # dist = dist.reshape(grid_x.shape)
            #grid_z[dist > mask_distance] = np.nan

        #grid_existing = clean_grid(grid_existing)
        #grid_proposed = clean_grid(grid_proposed)
        
        # --- Create 3D overlay plot ---

        #fig = go.FigureWidget()

        # Existing surface (base)
        #fig.add_trace(go.Surface(x=grid_x, y=grid_y, z=grid_existing,colorscale="Viridis",opacity=0.8,name="Existing Surface"))

        # Proposed surface (overlay, semi-transparent red)
        #fig.add_trace(go.Surface(x=grid_x, y=grid_y, z=grid_proposed,colorscale="Reds",opacity=0.5,name="Proposed Surface"))

        # Layout styling
        #fig.update_layout(scene=dict(xaxis_title="Easting (relative)",yaxis_title="Northing (relative)",zaxis_title="Elevation (ft)",aspectratio=dict(x=1, y=1, z=0.3)),height=700)

        #display(fig)
        

In [21]:
#User Inputs

#Prompt exisiting surface data file
input_csv_widget = widgets.FileUpload(accept = '.csv', multiple = False, description = 'Upload CSV File',   layout = widgets.Layout(width = '400px'))
input_csv_widget.style.description_width = '150px'

#Prompt minimum top-of-pile height
min_top_widget = widgets.FloatText(value=0.0, description = "Minimum top-of-pile height (ft):", step=0.01, layout = widgets.Layout(width = '275px'))
min_top_widget.style.description_width = '185px'

#prompt maximum top-of-pile height
max_top_widget = widgets.FloatText(value=0.0, description = "Maximum top-of-pile height (ft):", step=0.01, layout = widgets.Layout(width = '275px'))
max_top_widget.style.description_width = '195px'

#Prompt instilation tolerance
tolerance_widget = widgets.FloatText(value=0.0, description = "Tolerance (ft):", step=0.01, layout = widgets.Layout(width = '150px'))
tolerance_widget.style.description_width = '85px'

#Prompt Project name
project_name_widget = widgets.Text(placeholder = 'Enter project name', description = "Project name:", layout = widgets.Layout(width = '250px'))
project_name_widget.style.description_width = '79px'

#Prompt step size
step_size_slider = widgets.FloatSlider(value = 0.5, min = 0.01, max = 1.0, step = 0.01, description = 'step size', style = {'description_width': 'initial'}, layout = widgets.Layout(width = '300px'))
step_size_secondary_label = widgets.Label(value = "Lower step size -> More accurate but longer simulation")
step_size_third_label = widgets.Label(value = "Higher step size -> Less accurate but quicker simulation")

#Prompt max slope
max_slope_slider = widgets.FloatSlider(value = 15, min = 0, max = 89, step = 0.5, description = 'Maximum Slope', readout_format = '.1f', style = {'description_width': 'initial'}, layout = widgets.Layout(width = '300px'))
max_slope_slider_description = widgets.Label("Maximum slope typically 15°")

#create run and stop buttons for UI
run = widgets.Button(description = 'Run Analysis', button_style = 'success', tooltip = 'click to run', icon = 'play', layout = widgets.Layout(width = '200px', height = '40px'))
stop = widgets.Button(description = 'Reset', button_style = 'warning', tooltip = 'reset all inputs and clear results', icon = 'refresh', layout = widgets.Layout(width = '200px', height = '40px'))

#Create output area's
out = widgets.Output()
out2 = widgets.Output()
out3 = widgets.Output()
out4 = widgets.Output()
out5 = widgets.Output()
out6 = widgets.Output()
out7 = widgets.Output()
out8 = widgets.Output()
out9 = widgets.Output()
out10 = widgets.Output()
out11 = widgets.Output()
out12 = widgets.Output()

#lock in entered values
def run_next_cell(b):
    #Clear any previous values
    out.clear_output()
    global input_csv, min_top, max_top, tolerance, project_name, step_size, max_slope_deg, fig
    input_csv = input_csv_widget.value
    project_name = project_name_widget.value
    if input_csv_widget.value: 
        out.clear_output()
        out2.clear_output()
        out3.clear_output()
        out4.clear_output()
        out5.clear_output()
        out6.clear_output()
        out7.clear_output()
        out8.clear_output()
        out9.clear_output()
        out10.clear_output()
        out11.clear_output()
        out12.clear_output()
        uploaded_file = next(iter(input_csv_widget.value))
        content = uploaded_file['content']
        name = uploaded_file['name']

        with tempfile.NamedTemporaryFile(delete = False, suffix = ".csv") as tmp:
            tmp.write(content)
            tmp_path = tmp.name
        input_csv = tmp_path

        min_top = min_top_widget.value
        max_top = max_top_widget.value
        tolerance = tolerance_widget.value
        step_size = step_size_slider.value
        max_slope_deg = max_slope_slider.value
        with out:    
            print(f"\033[1mExisting surface data:\033[0m File found: '{name}'")

        #Run Analysis
        df, tracker_groups = load_data_and_create_trackers(input_csv, tolerance_widget)
        out.clear_output()
        final_df = run_full_pipeline(tracker_groups, min_top, max_top, tolerance, step_size, max_slope_deg)
        
        #Display Data
        with out:
            spinnerd = """<div style="display:flex;align-items:center;gap:10px;"><div class="loader" style="border: 4px solid #f3f3f3;border-top: 4px solid orange;border-radius: 50%;width: 18px;height: 18px;animation: spin 1s linear infinite;"></div><span style="color: orange; font-weight: bold;">Displaying Data...</span></div><style>@keyframes spin {0% { transform: rotate(0deg); }100% { transform: rotate(360deg); }}</style>"""
            p_data = widgets.HTML(value = spinnerd)
            display(p_data)
        output_path = export_csv(final_df, project_name)
        results = compare_surfaces(input_csv, project_name)
        p_data.value = """<style>@keyframes fadeOut {from {opacity:1;} to {opacity:0;}}@keyframes fadeIn {from {opacity:0;} to {opacity:1;}}</style><div id='success' style='animation:fadeIn 0.5s;'><span style='color:green;font-weight:bold;'>✔ Data Displayed!</span></div>"""
        df_exported = pd.read_csv(output_path)
        
        #Plot Heatmap
        with out:
            spinnerh = """<div style="display:flex;align-items:center;gap:10px;"><div class="loader" style="border: 4px solid #f3f3f3;border-top: 4px solid orange;border-radius: 50%;width: 18px;height: 18px;animation: spin 1s linear infinite;"></div><span style="color: orange; font-weight: bold;">Plotting Heatmap...</span></div><style>@keyframes spin {0% { transform: rotate(0deg); }100% { transform: rotate(360deg); }}</style>"""
            p_heatmap = widgets.HTML(value = spinnerh)
            display(p_heatmap)
        plot_heatmap(results["df_combined"],results["cut_volume_ft3"],results["fill_volume_ft3"],results["cut_volume_yd3"],results["fill_volume_yd3"])  
        p_heatmap.value = """<style>@keyframes fadeOut {from {opacity:1;} to {opacity:0;}}@keyframes fadeIn {from {opacity:0;} to {opacity:1;}}</style><div id='success' style='animation:fadeIn 0.5s;'><span style='color:green;font-weight:bold;'>✔ Heatmap Plotted!</span></div>"""

        #Plot Pile Heatmap
        with out:
            spinnerdh = """<div style="display:flex;align-items:center;gap:10px;"><div class="loader" style="border: 4px solid #f3f3f3;border-top: 4px solid orange;border-radius: 50%;width: 18px;height: 18px;animation: spin 1s linear infinite;"></div><span style="color: orange; font-weight: bold;">Plotting Pile Heatmap...</span></div><style>@keyframes spin {0% { transform: rotate(0deg); }100% { transform: rotate(360deg); }}</style>"""
            p_dotted_heatmap = widgets.HTML(value = spinnerdh)
            display(p_dotted_heatmap)
        dot_heatmap(results["df_combined"])
        p_dotted_heatmap.value = """<style>@keyframes fadeOut {from {opacity:1;} to {opacity:0;}}@keyframes fadeIn {from {opacity:0;} to {opacity:1;}}</style><div id='success' style='animation:fadeIn 0.5s;'><span style='color:green;font-weight:bold;'>✔ Pile Heatmap Plotted!</span></div>"""
        
        #Plot 3D Heatmap
        with out:
            spinner3h = """<div style="display:flex;align-items:center;gap:10px;"><div class="loader" style="border: 4px solid #f3f3f3;border-top: 4px solid orange;border-radius: 50%;width: 18px;height: 18px;animation: spin 1s linear infinite;"></div><span style="color: orange; font-weight: bold;">Plotting 3D Heatmap...</span></div><style>@keyframes spin {0% { transform: rotate(0deg); }100% { transform: rotate(360deg); }}</style>"""
            p_3d_heatmap = widgets.HTML(value = spinner3h)
            display(p_3d_heatmap)
        plot_change_in_terrain(results['df_combined'], value_col='DeltaElevation', x_col='Northing', y_col='Easting')
        p_3d_heatmap.value = """<style>@keyframes fadeOut {from {opacity:1;} to {opacity:0;}}@keyframes fadeIn {from {opacity:0;} to {opacity:1;}}</style><div id='success' style='animation:fadeIn 0.5s;'><span style='color:green;font-weight:bold;'>✔ 3D Heatmap Plotted!</span></div>"""

        #Plot 3D Existing Ground
        with out:
            spinner3e = """<div style="display:flex;align-items:center;gap:10px;"><div class="loader" style="border: 4px solid #f3f3f3;border-top: 4px solid orange;border-radius: 50%;width: 18px;height: 18px;animation: spin 1s linear infinite;"></div><span style="color: orange; font-weight: bold;">Plotting 3D Existing Ground...</span></div><style>@keyframes spin {0% { transform: rotate(0deg); }100% { transform: rotate(360deg); }}</style>"""
            p_3d_existing_ground = widgets.HTML(value = spinner3e)
            display(p_3d_existing_ground)
        plot_existing_surface(results['df_combined'], df_exported)
        p_3d_existing_ground.value = """<style>@keyframes fadeOut {from {opacity:1;} to {opacity:0;}}@keyframes fadeIn {from {opacity:0;} to {opacity:1;}}</style><div id='success' style='animation:fadeIn 0.5s;'><span style='color:green;font-weight:bold;'>✔ 3D Existing Ground Plotted!</span></div>"""

        #Plot 3D Proposed Ground
        with out:
            spinner3p = """<div style="display:flex;align-items:center;gap:10px;"><div class="loader" style="border: 4px solid #f3f3f3;border-top: 4px solid orange;border-radius: 50%;width: 18px;height: 18px;animation: spin 1s linear infinite;"></div><span style="color: orange; font-weight: bold;">Plotting 3D Proposed Ground...</span></div><style>@keyframes spin {0% { transform: rotate(0deg); }100% { transform: rotate(360deg); }}</style>"""
            p_3d_proposed_ground = widgets.HTML(value = spinner3p)
            display(p_3d_proposed_ground)
        plot_proposed_ground(df_exported, results['df_combined'])
        p_3d_proposed_ground.value = """<style>@keyframes fadeOut {from {opacity:1;} to {opacity:0;}}@keyframes fadeIn {from {opacity:0;} to {opacity:1;}}</style><div id='success' style='animation:fadeIn 0.5s;'><span style='color:green;font-weight:bold;'>✔ 3D Proposed Ground Plotted!</span></div>"""

        #Plot Cross Section
        with out:
            spinnerc = """<div style="display:flex;align-items:center;gap:10px;"><div class="loader" style="border: 4px solid #f3f3f3;border-top: 4px solid orange;border-radius: 50%;width: 18px;height: 18px;animation: spin 1s linear infinite;"></div><span style="color: orange; font-weight: bold;">Plotting Cross Section...</span></div><style>@keyframes spin {0% { transform: rotate(0deg); }100% { transform: rotate(360deg); }}</style>"""
            p_cross_section = widgets.HTML(value = spinnerc)
            display(p_cross_section)
        plot_side_view(results["df_combined"], df_exported)
        p_cross_section.value = """<style>@keyframes fadeOut {from {opacity:1;} to {opacity:0;}}@keyframes fadeIn {from {opacity:0;} to {opacity:1;}}</style><div id='success' style='animation:fadeIn 0.5s;'><span style='color:green;font-weight:bold;'>✔ Cross Section Plotted!</span></div>"""


        #Plot Existing Dotted Heatmap
        #with out:
        #    p_existing_dotted_heatmap = widgets.HTML(value = "<span style = 'color: orange;'>Plotting Existing Dotted Heatmap...</span>")
        #    display(p_existing_dotted_heatmap)
        #dot_heatmap_existing(results['df_combined'])
        #p_existing_dotted_heatmap.value = "<span style = 'color: green; font-weight: bold;'>Existing Dotted Heatmap Plotted!</span>"
        
        #Plot Proposed Dotted Heatmap
        #with out:
        #    p_proposed_dotted_heatmap = widgets.HTML(value = "<span style = 'color: orange;'>Plotting Proposed Dotted Heatmap...</span>")
        #    display(p_proposed_dotted_heatmap)
        #dot_heatmap_proposed(df_exported)
        #p_proposed_dotted_heatmap.value = "<span style = 'color: green; font-weight: bold;'>Proposed Dotted Heatmap Plotted!</span>"
        
        #Plot Existing vs Proposed
        #with out:
        #    p_ex_vs_pr = widgets.HTML(value = "<span style = 'color: orange;'>Plotting Existing vs Proposed...</span>")
        #    display(p_ex_vs_pr)
        #plot_existing_vs_proposed(dfp=df_exported, dfe=results['df_combined'])
        #p_ex_vs_pr.value = "<span style = 'color: green; font-weight: bold;'>Existing vs Proposed Plotted!</span>"

        #Clear all loading information
        out.clear_output()

            
    else:
        with out:
            print(f"\033[1mNo File Selected:\033[0m Please ensure you have uploaded a CSV file in the section above")
            
#Reset UI when Rest button is clicked
def reset(b):
    #Clear all outputs
    out.clear_output()
    out2.clear_output()
    out3.clear_output()
    out4.clear_output()
    out5.clear_output()
    out6.clear_output()
    out7.clear_output()
    out8.clear_output()
    out9.clear_output()
    out10.clear_output()
    out11.clear_output()
    out12.clear_output()
    with out:
        #Reset all inputs to orginal values
        input_csv_widget.set_trait('value', ())
        input_csv_widget._counter = 0
        project_name_widget.value = ''
        min_top_widget.value = 0
        max_top_widget.value = 0
        tolerance_widget.value = 0
        step_size_slider.value = 0.5
        max_slope_slider.value = 15

#Checks for if either UI buttons are clicked
run.on_click(run_next_cell)
stop.on_click(reset)

#Organizes UI into organized boxes
row1 = widgets.HBox([project_name_widget, input_csv_widget], layout = widgets.Layout(justify_content = 'center', grid_gap = '55px'))
row2 = widgets.HBox([min_top_widget, max_top_widget, tolerance_widget])
row3 = widgets.HBox([widgets.VBox([step_size_slider, step_size_secondary_label, step_size_third_label], 
                                  layout = widgets.Layout(border='1px solid #ccc', padding='12px', width='fit-content', align_items='center', grid_gap='12px')), 
                     widgets.VBox([max_slope_slider, max_slope_slider_description], 
                                  layout = widgets.Layout(border='1px solid #ccc', padding='12px', width='fit-content', align_items='center', grid_gap='12px'))],
                    layout = widgets.Layout(grid_gap = '12px', justify_content = 'center'))
row4 = widgets.HBox([run, stop], layout = widgets.Layout(justify_content = 'center', grid_gap = '100px'))
ui = widgets.VBox([row1, row2, row3, row4], layout = widgets.Layout(border='2px solid #ccc', padding='12px', width='fit-content', align_items='center', grid_gap='12px'))

#Organizes subtabs in Cross Section
tab2 = widgets.Tab(children = [out10, out12])
tab2.set_title(0, 'Northing')
tab2.set_title(1, 'Easting')

#Organizes output tabs
tab = widgets.Tab(children = [out2, out3, out7, out4, out6, out5, tab2])
tab.set_title(0, 'Data')
tab.set_title(1, 'Surface Heatmap')
tab.set_title(2, 'Pile Heatmap')
tab.set_title(3, '3D Heatmap')
tab.set_title(4, '3D Existing Surface')
tab.set_title(5, '3D Proposed Surface')
tab.set_title(6, 'Cross Section')
#tab.set_title(7, 'Exist. dotted heatmap')
#tab.set_title(8, 'Prop. dotted heatmap')
#tab.set_title(9, 'Existing vs Proposed')

#displays UI, loading data and output data
display(ui, out, tab)


Output()